<br>
<br>

## 1. 라이브러리 설치 및 import

In [1]:
"""
카카오 

네이티브 앱 키

REST API 키

JavaScript 키

Admin 키

"""

In [1]:
# 라이브러리의 버전을 맞춰주셔야 합니다. (특히 selenium은 꼭 버전을 맞춰 설치해주세요!)

!pip install beautifulsoup4==4.7.1
!pip install selenium==3.11.0

In [2]:
import urllib
import requests
import json
import datetime
import time
import re

from bs4 import BeautifulSoup 
from urllib.request import urlopen 

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import pandas as pd

import warnings
warnings.simplefilter("ignore", UserWarning)

<br>
<br>

## 2. 카카오 API 를 통한 요청 및 응답 엑셀파일 저장 (본문 중 일부)
> 아래의 2-2 및 2-3 에서 확인하실 수 있듯 **main_func 함수**를 호출하여 원하는 정보만 입력하면 저장된 엑셀 파일을 확인하실 수 있습니다.<br><br>
**한번에 너무 많은 페이지를 요청하고 저장하게되면 아래 3번 항목에서 전체 본문을 크롤링하고 저장할 때 무리가 있습니다.**<br>
적당하게 10 페이지 이내로 저장하고 저장된 엑셀 파일을 바탕으로 3번 항목에서 전체 본문을 크롤링해서 저장해보시며 페이지 수를 늘려보세요.


<br>

#### 2-1. 카카오 API 활용을 위한 함수 선언

In [3]:
MYAPP_KEY = 'd9bREST KEY를입력해주세요5f6d'

In [4]:
# 주어진 검색 대상(web/cafe/blog/tip/book) 및 키워드 등의 정보를 바탕으로 API 요청을 위한 URL을 구성합니다.
def gen_search_url(api_node, search_text, start_num, disp_num):
    
    if api_node == 'book':
        base = "https://dapi.kakao.com/v3/search"
    else:
        base = "https://dapi.kakao.com/v2/search"
    
    node = "/" + api_node + ".json"
    param_query = "?query=" + urllib.parse.quote(search_text)
    param_start = "&page=" + str(start_num)
    param_disp = "&size=" + str(disp_num)
    
    return base + node + param_query + param_start + param_disp


# 구성이 완료된 URL을 바탕으로 API 요청을 진행하고 json type(python dict)의 응답을 받아옵니다.
def get_result_onpage(url):
    
    request = urllib.request.Request(url)
    header = {'Authorization': 'KakaoAK {}'.format(MYAPP_KEY)}
    response = requests.post(url, headers=header)
    print("[%s] url Request Success" % datetime.datetime.now())
    
    return json.loads(response.text)


# 주어진 str에서 HTML 태그들을 모두 삭제하고, 그 외 불필요한 기호를 삭제합니다.
def delete_tag(input_str):
    pattern = re.compile(r'<.*?>') # HTML 태그들을 모두 삭제하기 위한 정규표현식 패턴
    input_str = pattern.sub('', input_str)
    input_str = input_str.replace("&lt;","")
    input_str = input_str.replace("&gt;","")
    input_str = input_str.replace("&amp;","")
    
    return input_str


# 주어진 json type의 API 응답 데이터에서 웹페이지별 전체 데이터를 꺼내어 DataFrame에 담는 함수입니다.
def doc_to_df(search_target, json_data):
    
    # 'tip' 의 경우는 응답으로 주어지는 URL이 질문과 답변에 대한 2가지 URL입니다.
    if search_target == 'tip':
        titles = [delete_tag(each['title']) for each in json_data['documents']]
        contents = [delete_tag(each['contents']) for each in json_data['documents']]    
        q_urls = [each['q_url'] for each in json_data['documents']]
        a_urls = [each['a_url'] for each in json_data['documents']]        
        datetimes = [each['datetime'][:10] + ', ' + each['datetime'][11:19] for each in json_data['documents']]
        
        result_pd = pd.DataFrame({'titles':titles,
                                  'contents':contents, 
                                  'urls_question':q_urls, 
                                  'urls_answer':a_urls, 
                                  'datetimes':datetimes}, 
                                 columns=['titles', 'contents', 'urls_question', 'urls_answer', 'datetimes'])
    
    else:
        titles = [delete_tag(each['title']) for each in json_data['documents']]
        contents = [delete_tag(each['contents']) for each in json_data['documents']]    
        urls = [each['url'] for each in json_data['documents']]
        datetimes = [each['datetime'][:10] + ', ' + each['datetime'][11:19] for each in json_data['documents']]
        
        result_pd = pd.DataFrame({'titles':titles,
                                  'contents':contents, 
                                  'urls':urls, 
                                  'datetimes':datetimes}, 
                                 columns=['titles', 'contents', 'urls', 'datetimes'])
    return result_pd


# 주어진 검색 대상(web/cafe/blog/tip/book) 및 키워드 등의 정보를 바탕으로 크롤링을 진행하는 메인 함수입니다.
def crawl_all(search_target, keyword, max_page, doc_per_page):
    
    # 첫번째 페이지에 대한 결과 데이터로 base dataframe을 생성합니다.
    url = gen_search_url(search_target, keyword, 1, doc_per_page) 
    one_result = get_result_onpage(url)
    main_df = doc_to_df(search_target, one_result)
    
    # 두번째 페이지부터 크롤링 후 DataFrame merge 작업을 진행하며, 마지막 페이지까지만 크롤링 후 함수를 종료합니다.
    for page in range(2, max_page+1):
        url = gen_search_url(search_target, keyword, page, doc_per_page)
        print('Getting data from : {}'.format(url))
        one_result = get_result_onpage(url)
        
        if one_result['meta']['is_end'] == True:
            print('\nURL Response의 (meta)is_end 값이 True입니다. 마지막 페이지인 현재 페이지까지만 수합합니다.')
            temp_df = doc_to_df(search_target, one_result)
            main_df = pd.concat([main_df, temp_df], axis=0)
            
            return main_df
    
        else:
            temp_df = doc_to_df(search_target, one_result)
            main_df = pd.concat([main_df, temp_df], axis=0)

        # 단 시간 내의 빈번한 요청으로 인해 API 요청이 막히는 것을 방지하기 위한 sleep입니다.
        time.sleep(5)
    
    return main_df
    
    
# 사용자로부터 크롤링을 위한 정보를 받아들이고 메인 크롤링 함수를 실행한 후 결과 DataFrame을 엑셀 파일로 저장합니다.
# 엑셀 파일의 형식은 'result_크롤링분야_연월일_시분.xlsx' 입니다.
# 엑셀 파일을 열었을 때 url 텍스트들로 인해 경고 메시지가 뜰 경우 복구 "예"를 클릭하시고 그대로 사용하시면 됩니다. (문제 없음)
def main_func():

    search_target = input('문서를 크롤링할 대상을 입력해주세요 (web/cafe/blog/tip/book 중 택일, 소문자) : ')
    keyword = input('크롤링을 원하시는 키워드를 입력해주세요 : ')
    max_page = input('크롤링을 원하시는 최대 페이지를 2~50 사이의 숫자로 입력해주세요. : ')
    doc_per_page = input('크롤링을 원하시는 페이지 당 문서 수를 1~50 사이의 숫자로 입력해주세요. : ')

    if (search_target=='') or (keyword=='') or (max_page=='') or (doc_per_page==''):
        print('\n입력된 값 중 제대로 입력되지 않은 값이 있습니다. 다시 실행해주세요.')
        return False
    
    if search_target in ['web', 'book']:
        print('\n[ web ] 혹은 [ book ] 을 대상으로 크롤링 시 웹사이트 중 URL이 255자가 넘어가는 사이트는 URL이 엑셀에 저장이 되지 않습니다.')
        print('함수 실행 후 돌려받은 DataFrame에는 그러한 웹사이트의 URL이 그대로 남아있으므로 DataFrame에서 확인하실 수 있습니다.\n')
              
    max_page = int(max_page)
    doc_per_page = int(doc_per_page)
    
    result_df = crawl_all(search_target, keyword, max_page, doc_per_page)
    result_df = result_df.reset_index()
    del result_df['index']
    print('\nCrawling process is finished!')
    
    file_name = 'result_{}_{}.xlsx'.format(search_target, datetime.datetime.now().strftime('%y%m%d_%H%M'))
    result_df.to_excel(file_name, encoding='utf-8', index=False)
    print('\nCrawling result is saved at [ {} ]'.format(file_name))
    
    return result_df

<br>

#### 2-2. 카카오 API 활용을 통한 데이터 요청 및 엑셀 저장 예시 (cafe / 뇌과학 / 3페이지 / 페이지당 50개 문서)

In [9]:
result = main_func()
result.head()

문서를 크롤링할 대상을 입력해주세요 (web/cafe/blog/tip/book 중 택일, 소문자) : book
크롤링을 원하시는 키워드를 입력해주세요 : 창의성
크롤링을 원하시는 최대 페이지를 2~50 사이의 숫자로 입력해주세요. : 50
크롤링을 원하시는 페이지 당 문서 수를 1~50 사이의 숫자로 입력해주세요. : 50

[ web ] 혹은 [ book ] 을 대상으로 크롤링 시 웹사이트 중 URL이 255자가 넘어가는 사이트는 URL이 엑셀에 저장이 되지 않습니다.
함수 실행 후 돌려받은 DataFrame에는 그러한 웹사이트의 URL이 그대로 남아있으므로 DataFrame에서 확인하실 수 있습니다.

[2019-08-06 19:32:48.820631] url Request Success
Getting data from : https://dapi.kakao.com/v3/search/book.json?query=%EC%B0%BD%EC%9D%98%EC%84%B1&page=2&size=50
[2019-08-06 19:32:48.935861] url Request Success
Getting data from : https://dapi.kakao.com/v3/search/book.json?query=%EC%B0%BD%EC%9D%98%EC%84%B1&page=3&size=50
[2019-08-06 19:32:54.064013] url Request Success
Getting data from : https://dapi.kakao.com/v3/search/book.json?query=%EC%B0%BD%EC%9D%98%EC%84%B1&page=4&size=50
[2019-08-06 19:32:59.212581] url Request Success
Getting data from : https://dapi.kakao.com/v3/search/book.json?query=%EC%B0%BD%EC%9D%98%EC%84%B1&page=5&size=50
[2019-08-06 1

,titles,contents,urls,datetimes
0,인문학으로 광고하다,『인문학으로 광고하다』는 창의성에 관한 책이다. 이 책은 창의성에 대해 이야기하기 ...,https://search.daum.net/search?w=bookpage&book...,"2009-08-27, 00:00:00"
1,창의성을 지휘하라,『창의성을 지휘하라』는 기업의 대표적 롤모델인 픽사와 디즈니 애니메이션의 성공신화를...,https://search.daum.net/search?w=bookpage&book...,"2014-09-16, 00:00:00"
2,창의성,"『창의성』은 총 10장으로 나누어 구성되어 있으며, 윤리학과 창의성의 개념을 설명하...",https://search.daum.net/search?w=bookpage&book...,"2012-02-29, 00:00:00"
3,창의성,,https://search.daum.net/search?w=bookpage&book...,"2005-03-07, 00:00:00"
4,창의성의 즐거움(양장본 HardCover),,https://search.daum.net/search?w=bookpage&book...,"2003-11-28, 00:00:00"


<br>
<br>
<br>

## 3. 본문 중 일부가 저장된 엑셀 파일을 바탕으로 전체 데이터를 크롤링하는 코드 
> 위 main_func 함수를 실행하여 본문 중 일부가 포함된 데이터를 엑셀 파일로 저장해두고 이를 불러와야 합니다.<br><br>
아래 3-3 ~ 3-6 을 확인하시면 실행 예시를 확인하실 수 있으며, <br>
**위 2번 항목에서 저장한 파일을 crawl_full_contents 함수에게 전달**하면 <br>
알아서 크롤링이 가능한 항목들을 전체 크롤링한 다음 **fullresult~~~ 이름의 엑셀 파일로 저장**해줍니다.<br>
('web'은 각 사이트의 구조가 다르므로 일괄적인 웹크롤링이 불가능합니다.)<br><br>
한번에 너무 많은 양의 문서를 크롤링하게되면 저장된 엑셀 파일이 열리지 않거나 DataFrame이 memory 관련 에러를 발생시킬 수 있습니다. <br>
memory 관련 에러가 발생하면 다시 위 2번 항목으로 돌아가 보다 작은 페이지수를 입력하여 main_func 함수를 실행하고 <br>
새롭게 저장된 엑셀파일을 활용해 전체 본문 크롤링을 실행해보세요.

<br>

#### 3-1. URL 을 바탕으로 전체 본문을 크롤링하기 위한 함수 선언 (cafe, blog, tip, book 각각에 대하여)

In [17]:
def full_cafe(dataframe):
    
    driver = webdriver.Chrome(executable_path='/Users/spark/Downloads/190806_Daum/chromedriver.exe') 
    # driver = webdriver.PhantomJS('(driver) phantomjs.exe')

    full_contents = []
    for index, url in enumerate(dataframe['urls']):    
        
        if index % 10 == 0:
            print('{}번째 항목을 크롤링 중입니다 (URL : {})'.format(index, url))
        
        if url != 'no_url':
            url = url[:7] + 'm.' + url[7:]
            driver.get(url)
            time.sleep(2)
            web_page = BeautifulSoup(driver.page_source, 'html.parser')

            content = web_page.find('p', {'class':'cafe-editor-text'})
            if content != None:
                full_content = content.get_text()
                full_content = full_content.replace("\n", "")
                full_content = full_content.strip()
            else:
                full_content = '회원가입이 필요한 까페입니다.'
        else:
            full_content = '까페의 URL이 255자를 초과하여 엑셀파일에서 제외되었습니다.'
            
        full_contents.append(full_content)
    
    driver.close()
    driver.quit()
    
    dataframe['full_contents'] = full_contents
    print('\nCrawling for full-contents is finished!')
    
    return dataframe

In [18]:
def full_blog(dataframe):
    
    full_contents = []
    for index, url in enumerate(dataframe['urls']):    

        if index % 10 == 0:
            print('{}번째 항목을 크롤링 중입니다 (URL : {})'.format(index, url))

        if 'tistory.com' in url:
            div_class = 'tt_article_useless_p_margin'
        elif 'brunch.co.kr' in url:
            div_class = 'wrap_body'
        elif 'blog.naver.com' in url:
            div_class = 'se-main-container'
        else:
            div_class = None
        
        if div_class != None:
            try:
                response = urlopen(url)
                source = BeautifulSoup(response, 'html.parser')
                article = source.find('div', {'class':div_class})
            except:
                print('다음 항목 크롤링 중 빈번한 요청에 의해 한시적으로 요청이 막혔으나 나머지 항목을 대상으로 크롤링을 재개합니다. {}'.format(url))
        else:
            article = None

        if article != None: 
            full_content = article.get_text()
            full_content = full_content.replace("\n", "")
            full_content = full_content.strip()
        else:
            full_content = '본문이 존재하지 않거나 크롤링이 불가능한 블로그 글입니다.'

        full_contents.append(full_content)

    dataframe['full_contents'] = full_contents
    print('\nCrawling for full-contents is finished!')

    return dataframe

In [19]:
def full_tip(dataframe):
    
    driver = webdriver.Chrome(executable_path='/Users/spark/Downloads/190806_Daum/chromedriver.exe') 
    # driver = webdriver.PhantomJS('(driver) phantomjs.exe')

    full_questions = []
    full_answers = []
    
    for index, url in enumerate(dataframe['urls_question']):    
        
        if index % 10 == 0:
            print('{}번째 항목을 크롤링 중입니다 (URL : {})'.format(index, url))
        
        url = url[:7] + 'm.' + url[7:]
        driver.get(url)
        time.sleep(2)
        web_page = BeautifulSoup(driver.page_source, 'html.parser')
        
        full_question = web_page.find('div', {'class':'question_cont'}).find('div', {'class':'txt_collect'})
        if full_question != None:
            full_question = full_question.get_text()
            full_question = full_question.replace("\n", "")
            full_question = full_question.strip()
        else:
            full_question = '질문이 발견되지 않았습니다. (오픈지식에 해당합니다.)'

        full_answer = web_page.find('div', {'id':'answerList'})
        if full_answer != None:
            full_answer = full_answer.find_all('div', {'class':'txt_collect'})
            full_answer = ' '.join([answer.get_text().replace('\n', '').strip() for answer in full_answer])
        else:
            full_answer = '답변이 발견되지 않았습니다.'

        full_questions.append(full_question)
        full_answers.append(full_answer)
    
    driver.close()
    driver.quit()
    
    dataframe['full_questions'] = full_questions
    dataframe['full_answers'] = full_answers
    print('\nCrawling for full-contents is finished!')
    
    return dataframe

In [20]:
def full_book(dataframe):

    driver = webdriver.Chrome(executable_path='/Users/spark/Downloads/190806_Daum/chromedriver.exe') 
    # driver = webdriver.PhantomJS('(driver) phantomjs.exe')

    full_bookinfos = []
    for index, url in enumerate(dataframe['urls']):    

        if index % 10 == 0:
            print('{}번째 항목을 크롤링 중입니다 (URL : {})'.format(index, url))
        
        if url != 'no_url':
            url = url[:8] + 'm.' + url[8:]
            driver.get(url)
            time.sleep(2)
            web_page = BeautifulSoup(driver.page_source, 'html.parser')

            book_info_all = web_page.find_all('div', {'class':'info_desc'}) # 소개, 저자, 목차, 출판사서평

            # 현재 코드에는 [소개, 저자, 목차, 출판사서평] 중 '소개'만 따내어 저장하도록 구현해두었습니다.
            if book_info_all != None:
                full_bookinfo = book_info_all[0].get_text()
                full_bookinfo = full_bookinfo.replace("\n", "")
                full_bookinfo = full_bookinfo.strip()
            else:
                full_bookinfo = '책의 정보가 발견되지 않았습니다.'
        else:
            full_bookinfo = '책의 정보가 발견되지 않았습니다.'
            
        full_bookinfos.append(full_bookinfo)

    driver.close()
    driver.quit()

    dataframe['full_bookinfos'] = full_bookinfos
    print('\nCrawling for full-contents is finished!')

    return dataframe

<br>

#### 3-2. 위 2번 항목에서 저장한 [ 본문 일부 포함 ] 엑셀 파일을 받아들여 파일 이름에 따라 cafe/blog/tip/book 중 해당 항목 크롤러를 실행하는 함수 선언

In [21]:
def crawl_full_contents(file_name):
    
    if 'web' in file_name:
        print('[ web ]의 경우는 각 항목마다 웹사이트가 다르므로 본문 웹크롤링 적용이 불가능합니다.')
        return False
    
    elif 'cafe' in file_name:
        search_target = 'cafe'
        original_df = pd.read_excel(file_name, encoding='utf-8')
        original_df['urls'] = original_df['urls'].fillna('no_url')
        full_df = full_cafe(original_df)
        
    elif 'blog' in file_name:
        search_target = 'blog'
        original_df = pd.read_excel(file_name, encoding='utf-8')
        original_df['urls'] = original_df['urls'].fillna('no_url')
        full_df = full_blog(original_df)
        
    elif 'tip' in file_name:
        search_target = 'tip'
        original_df = pd.read_excel(file_name, encoding='utf-8')
        original_df['urls_question'] = original_df['urls_question'].fillna('no_url')
        full_df = full_tip(original_df)
        
    elif 'book' in file_name:
        search_target = 'book'
        original_df = pd.read_excel(file_name, encoding='utf-8')
        original_df['urls'] = original_df['urls'].fillna('no_url')
        full_df = full_book(original_df)
        
    else:
        print('\n파일명을 확인해주세요! 파일 이름 안에 [ web / cafe / blog / tip / book ] 중 하나의 단어가 들어있어야 합니다.')
        print('파일명 확인 후 다시 함수를 실행해주세요.')
        return False

    file_name = 'fullresult_{}_{}.xlsx'.format(search_target, datetime.datetime.now().strftime('%y%m%d_%H%M'))
    full_df.to_excel(file_name, encoding='utf-8', index=False)
    print('\nFull-crawling result is saved at [ {} ]'.format(file_name))
    
    return full_df

<br>

#### 3-3. [본문 일부 포함] 엑셀 파일을 바탕으로 전체 본문 크롤링 및 엑셀파일 저장 예시 (cafe)
> 회원가입 없이 바로 본문을 확인할 수 있는 까페 글들만 크롤링 가능합니다.

In [23]:
full_result = crawl_full_contents('result_blog_190806_1930_s.xlsx')
full_result.head()

0번째 항목을 크롤링 중입니다 (URL : http://blog.naver.com/PostView.nhn?blogId=autumnsk&logNo=221575363525)
10번째 항목을 크롤링 중입니다 (URL : https://brunch.co.kr/@inmunart/636)
20번째 항목을 크롤링 중입니다 (URL : https://brunch.co.kr/@yshinb/28)

Crawling for full-contents is finished!

Full-crawling result is saved at [ fullresult_blog_190806_2017.xlsx ]


,titles,contents,urls,datetimes,full_contents
0,우리 아이 창의성 키우는 방법,"아이들은 손을 문지르고, 얼굴도장을 찍기도 했다. 흥건해진 바닥 위로 폴짝폴짝 뛰다...",http://blog.naver.com/PostView.nhn?blogId=autu...,"2019-07-01, 23:02:00","© jcao329, 출처 Unsplash 아이들을 초등학교 입학 전에 놀이 위주..."
1,창의성이라는 유령,"꼰대가 되겠구나&#39;, 하고요. 어린 민지에게 무례한 사람 대처법이, 제게는 &...",http://free2world.tistory.com/1975,"2019-02-27, 06:35:00",본문이 존재하지 않거나 크롤링이 불가능한 블로그 글입니다.
2,철학과 창의성,지식보다는 창의성을 논하는 시대가 다가왔다. &#39;Idea&#39;가 문뜩 떠 ...,https://brunch.co.kr/@inmunart/402,"2019-02-02, 17:45:00",지식보다는 창의성을 논하는 시대가 다가왔다.'Idea'가 문뜩 떠 올랐다고 말들을 ...
3,&#34;창의성&#34;에 관하여,비슷한 경험이 있을 것이다. 그렇게 몰두하던 주제에 대한 해결책이 전혀 예기치 않던...,https://brunch.co.kr/@umcine/44,"2018-12-16, 00:28:00",중요한 PT를 앞에 두고 주제를 어떻게 잡고 접근할지를 고민만 하는데 날짜는 다가오...
4,[성품 교육] 창의성에 대하여,"손맛 따라 적당히 간 맞추듯 내 아이에 맞게, 내 걸음에 맞게 적당히 간 맞춰주세요...",http://yummystudy.tistory.com/426,"2018-07-14, 13:41:00",본문이 존재하지 않거나 크롤링이 불가능한 블로그 글입니다.


<br>

#### etc. 다음 웹사이트 로그인이 필요할 경우에는 아래 코드를 활용하세요

In [24]:
driver.get('https://logins.daum.net/accounts/loginform.do?mobilefull=1&category=cafe&url=http%3A%2F%2Fm.cafe.daum.net%2F_myCafe%3Fnull') 
time.sleep(3) 

driver.find_element_by_xpath("""//*[@id="mArticle"]/div/div/div/div[2]/a[2]""").click()
time.sleep(3) 

driver.find_element_by_xpath("""//*[@id="id"]""").send_keys('본인의 Daum 아이디') # id 
driver.find_element_by_xpath("""//*[@id="inputPwd"]""").send_keys('본인의 Daum 패스워드') # 패스워드 
driver.find_element_by_xpath("""//*[@id="loginBtn"]""").click() # 입력 버튼 클릭 
time.sleep(3)

NameError: name 'driver' is not defined